# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))
print(sentences[1])
print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counter = Counter(text)
    vocab = sorted(counter, key=counter.get, reverse=True)
    vocab_to_int = {word: i for i, word in enumerate(vocab,1)}
    int_to_vocab = {i: word for i, word in enumerate(vocab,1)}
    
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    dic={'.':'<PERIOD>',',':'<COMA>','"':'<QUOTE_MARK>', ';':'<SEMI>', '!':'<EXCLAMATION>','?':'<QUESTION>', '(':'<LEFT_PARENT>',')':'<RIGHT_PARENT>','--':'<DASH>','\n':'<RETURN>'}
    return dic

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input= tf.placeholder(tf.int32, [None,None], name='input')
    Targets = tf.placeholder(tf.int32,[None,None], name='targets')
    LearningRate=tf.placeholder(tf.float32, name='lr')
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    Cell = tf.contrib.rnn.MultiRNNCell([lstm for _ in range(rnn_size)])
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState=tf.identity(InitialState, name='initial_state')
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function

    return tf.contrib.layers.embed_sequence(input_data, vocab_size=vocab_size, embed_dim=embed_dim)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
        
    Outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(state, name='final_state')
    
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
   
    embeded= get_embed(input_data, vocab_size, embed_dim)
    output, FinalState = build_rnn(cell, embeded)
    Logits = tf.contrib.layers.fully_connected(output,num_outputs=vocab_size, activation_fn=None) 
    
    
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = (len(int_text))//(batch_size * seq_length)    
    int_text = int_text[:n_batches * batch_size * seq_length + 1]
    
    input_seq = [int_text[i*seq_length:i*seq_length+seq_length] for i in range(n_batches * batch_size)]
    int_text = int_text[1:]
    input_targets = [int_text[i*seq_length:i*seq_length+seq_length] for i in range(n_batches * batch_size)]
    
    batches = []
    
    for batch_n in range(n_batches):
        inputs = []
        targets = []
        for batch_index in range(batch_size):
            inputs.append(input_seq[batch_index * n_batches + batch_n])
            targets.append(input_targets[batch_index * n_batches + batch_n])
            
        batches.append([inputs, targets])
    batches[n_batches-1][1][batch_size-1][seq_length-1]= input_seq[0][0]
        
    return np.array(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)


Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [45]:
# Number of Epochs
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 10
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 1
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/53   train_loss = 8.822
Epoch   0 Batch    5/53   train_loss = 12.457
Epoch   0 Batch   10/53   train_loss = 13.647
Epoch   0 Batch   15/53   train_loss = 13.095
Epoch   0 Batch   20/53   train_loss = 12.206
Epoch   0 Batch   25/53   train_loss = 11.806
Epoch   0 Batch   30/53   train_loss = 10.809
Epoch   0 Batch   35/53   train_loss = 11.304
Epoch   0 Batch   40/53   train_loss = 10.077
Epoch   0 Batch   45/53   train_loss = 10.430
Epoch   0 Batch   50/53   train_loss = nan
Epoch   1 Batch    2/53   train_loss = 7.473
Epoch   1 Batch    7/53   train_loss = 7.615
Epoch   1 Batch   12/53   train_loss = 7.311
Epoch   1 Batch   17/53   train_loss = 7.577
Epoch   1 Batch   22/53   train_loss = 7.064
Epoch   1 Batch   27/53   train_loss = 7.520
Epoch   1 Batch   32/53   train_loss = 6.747
Epoch   1 Batch   37/53   train_loss = 6.804
Epoch   1 Batch   42/53   train_loss = 6.607
Epoch   1 Batch   47/53   train_loss = 6.715
Epoch   1 Batch   52/53   train_loss = 6.681
Epo

Epoch  17 Batch   14/53   train_loss = 6.005
Epoch  17 Batch   19/53   train_loss = 5.980
Epoch  17 Batch   24/53   train_loss = 5.987
Epoch  17 Batch   29/53   train_loss = 5.876
Epoch  17 Batch   34/53   train_loss = 6.042
Epoch  17 Batch   39/53   train_loss = 6.113
Epoch  17 Batch   44/53   train_loss = 5.984
Epoch  17 Batch   49/53   train_loss = 6.304
Epoch  18 Batch    1/53   train_loss = 5.685
Epoch  18 Batch    6/53   train_loss = 5.960
Epoch  18 Batch   11/53   train_loss = 5.977
Epoch  18 Batch   16/53   train_loss = 5.986
Epoch  18 Batch   21/53   train_loss = 5.964
Epoch  18 Batch   26/53   train_loss = 6.070
Epoch  18 Batch   31/53   train_loss = 6.030
Epoch  18 Batch   36/53   train_loss = 5.989
Epoch  18 Batch   41/53   train_loss = 6.142
Epoch  18 Batch   46/53   train_loss = 6.230
Epoch  18 Batch   51/53   train_loss = 6.385
Epoch  19 Batch    3/53   train_loss = 6.045
Epoch  19 Batch    8/53   train_loss = 5.909
Epoch  19 Batch   13/53   train_loss = 6.149
Epoch  19 

Epoch  34 Batch   28/53   train_loss = 5.862
Epoch  34 Batch   33/53   train_loss = 6.004
Epoch  34 Batch   38/53   train_loss = 6.058
Epoch  34 Batch   43/53   train_loss = 6.063
Epoch  34 Batch   48/53   train_loss = 6.151
Epoch  35 Batch    0/53   train_loss = 5.756
Epoch  35 Batch    5/53   train_loss = 5.955
Epoch  35 Batch   10/53   train_loss = 6.046
Epoch  35 Batch   15/53   train_loss = 6.012
Epoch  35 Batch   20/53   train_loss = 5.982
Epoch  35 Batch   25/53   train_loss = 5.957
Epoch  35 Batch   30/53   train_loss = 5.963
Epoch  35 Batch   35/53   train_loss = 6.154
Epoch  35 Batch   40/53   train_loss = 5.974
Epoch  35 Batch   45/53   train_loss = 6.086
Epoch  35 Batch   50/53   train_loss = nan
Epoch  36 Batch    2/53   train_loss = 5.956
Epoch  36 Batch    7/53   train_loss = 5.941
Epoch  36 Batch   12/53   train_loss = 6.050
Epoch  36 Batch   17/53   train_loss = 6.018
Epoch  36 Batch   22/53   train_loss = 5.875
Epoch  36 Batch   27/53   train_loss = 6.033
Epoch  36 Ba

Epoch  51 Batch   42/53   train_loss = 5.981
Epoch  51 Batch   47/53   train_loss = 6.121
Epoch  51 Batch   52/53   train_loss = 6.176
Epoch  52 Batch    4/53   train_loss = 5.987
Epoch  52 Batch    9/53   train_loss = 5.844
Epoch  52 Batch   14/53   train_loss = 5.939
Epoch  52 Batch   19/53   train_loss = 5.886
Epoch  52 Batch   24/53   train_loss = 5.910
Epoch  52 Batch   29/53   train_loss = 5.783
Epoch  52 Batch   34/53   train_loss = 5.955
Epoch  52 Batch   39/53   train_loss = 6.026
Epoch  52 Batch   44/53   train_loss = 5.927
Epoch  52 Batch   49/53   train_loss = 6.225
Epoch  53 Batch    1/53   train_loss = 5.751
Epoch  53 Batch    6/53   train_loss = 5.894
Epoch  53 Batch   11/53   train_loss = 5.914
Epoch  53 Batch   16/53   train_loss = 5.915
Epoch  53 Batch   21/53   train_loss = 5.881
Epoch  53 Batch   26/53   train_loss = 5.990
Epoch  53 Batch   31/53   train_loss = 5.948
Epoch  53 Batch   36/53   train_loss = 5.920
Epoch  53 Batch   41/53   train_loss = 6.061
Epoch  53 

Epoch  69 Batch    3/53   train_loss = 5.986
Epoch  69 Batch    8/53   train_loss = 5.856
Epoch  69 Batch   13/53   train_loss = 6.093
Epoch  69 Batch   18/53   train_loss = 5.872
Epoch  69 Batch   23/53   train_loss = 5.868
Epoch  69 Batch   28/53   train_loss = 5.826
Epoch  69 Batch   33/53   train_loss = 5.957
Epoch  69 Batch   38/53   train_loss = 6.008
Epoch  69 Batch   43/53   train_loss = 6.026
Epoch  69 Batch   48/53   train_loss = 6.113
Epoch  70 Batch    0/53   train_loss = 5.846
Epoch  70 Batch    5/53   train_loss = 5.937
Epoch  70 Batch   10/53   train_loss = 6.022
Epoch  70 Batch   15/53   train_loss = 5.976
Epoch  70 Batch   20/53   train_loss = 5.930
Epoch  70 Batch   25/53   train_loss = 5.905
Epoch  70 Batch   30/53   train_loss = 5.913
Epoch  70 Batch   35/53   train_loss = 6.099
Epoch  70 Batch   40/53   train_loss = 5.943
Epoch  70 Batch   45/53   train_loss = 6.056
Epoch  70 Batch   50/53   train_loss = nan
Epoch  71 Batch    2/53   train_loss = 5.952
Epoch  71 Ba

Epoch  86 Batch   17/53   train_loss = 5.958
Epoch  86 Batch   22/53   train_loss = 5.838
Epoch  86 Batch   27/53   train_loss = 5.977
Epoch  86 Batch   32/53   train_loss = 5.834
Epoch  86 Batch   37/53   train_loss = 6.063
Epoch  86 Batch   42/53   train_loss = 5.955
Epoch  86 Batch   47/53   train_loss = 6.090
Epoch  86 Batch   52/53   train_loss = 6.146
Epoch  87 Batch    4/53   train_loss = 5.976
Epoch  87 Batch    9/53   train_loss = 5.832
Epoch  87 Batch   14/53   train_loss = 5.923
Epoch  87 Batch   19/53   train_loss = 5.853
Epoch  87 Batch   24/53   train_loss = 5.886
Epoch  87 Batch   29/53   train_loss = 5.751
Epoch  87 Batch   34/53   train_loss = 5.925
Epoch  87 Batch   39/53   train_loss = 5.998
Epoch  87 Batch   44/53   train_loss = 5.910
Epoch  87 Batch   49/53   train_loss = 6.184
Epoch  88 Batch    1/53   train_loss = 5.787
Epoch  88 Batch    6/53   train_loss = 5.877
Epoch  88 Batch   11/53   train_loss = 5.900
Epoch  88 Batch   16/53   train_loss = 5.893
Epoch  88 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [50]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [51]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    index=np.argmax(probabilities)
    word=int_to_vocab[index]
    
    return word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [52]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:........................................................................................................................................................................................................


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.